#1. Funciones auxiliares y lectura de datos


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

def leer_rango_excel_a_array(ruta_archivo, hoja_nombre=0, columnas_a_leer="", filas_a_leer=1):
    """
    Abre un archivo Excel y extrae un rango de celdas específico
    (D3 a BO42) y lo guarda en un array (lista de listas).
"""
    print(f"Intentando leer el archivo: {ruta_archivo}...")

    try:
        # 1. Filas:
        # Queremos empezar en la Fila 3.
        # Las filas en pandas se indexan desde 0.
        # Fila 1 = índice 0, Fila 2 = índice 1, Fila 3 = índice 2.
        # Así que saltamos las 2 primeras filas.
        filas_a_saltar = 3  # (Saltamos Fila 1 :texto Fila 2 : calibracion y Fila 3: checkeo salon vs dormitorio)


        #filas_a_leer = 40
        # Queremos leer desde la fila 3 hasta la 42 (inclusive).
        # Número de filas = 42 - 3 + 1 = 40 filas.

        # --- Lectura del archivo ---

        # Usamos 'header=None' porque el rango (D3) no es un encabezado,
        # es el inicio de los datos.
        df_rango = pd.read_excel(
            ruta_archivo,
            sheet_name= hoja_nombre, #Segunda hoja, Espectro Total
            header=None,             # Trata la primera fila leída (Fila 3) como datos
            skiprows=filas_a_saltar, # Salta las 2 primeras filas
            usecols=columnas_a_leer, # Lee solo el rango de columnas D-BO
            nrows=filas_a_leer       # Lee exactamente 40 filas
        )

        # Convertir el DataFrame de pandas a un array (lista de listas)
        # .values obtiene los datos como un array de NumPy
        # .tolist() lo convierte a una lista estándar de Python
        array_valores = df_rango.values.tolist()

        # Verificación
        if array_valores:
            num_filas = len(array_valores)
            num_cols = len(array_valores[0]) # Longitud de la primera fila
            print(f"Éxito: Se leyó un array de {num_filas} filas x {num_cols} columnas.")
        else:
            print("Advertencia: El rango D3:BO42 parece estar vacío.")

        return array_valores

    except FileNotFoundError:
        print(f"Error: No se pudo encontrar el archivo en la ruta: '{ruta_archivo}'.")
        return None
    except Exception as e:
        print(f"Ocurrió un error inesperado al leer el rango: {e}")
        return None

def separar_filas_en_matrices(datos_array):
    """
    Toma una lista de listas (matriz) y la separa en dos matrices:
    - Una con las filas de índice par (0, 2, 4...)
    - Otra con las filas de índice impar (1, 3, 5...)
    """

    # Listas para guardar las *filas* (que son listas)
    matriz_filas_impares_excel = []
    matriz_filas_pares_excel = []

    for i, fila in enumerate(datos_array):

        # i % 2 == 0 significa índice 0, 2, 4... (Filas Excel 3, 5, 7...)
        if i % 2 == 0:
            # --- CAMBIO CLAVE ---
            # Añade la 'fila' (que es una lista) como un nuevo elemento
            matriz_filas_impares_excel.append(fila)

        # else significa índice 1, 3, 5... (Filas Excel 4, 6, 8...)
        else:
            # --- CAMBIO CLAVE ---
            matriz_filas_pares_excel.append(fila)

    return matriz_filas_impares_excel, matriz_filas_pares_excel


In [52]:
#LO PRIMERO: ABRIR LOS ARCHIVOS

archivo_S1= "P3_G7_S1_Datos.xlsx"
archivo_S2= "P3_G7_S2_Datos.xlsx"

datos_sonometro_1 = leer_rango_excel_a_array(archivo_S1, "Banda Ancha Total", "U" , 24)
datos_sonometro_2 = leer_rango_excel_a_array(archivo_S2, "Banda Ancha Total", "U" , 48)

#SEPARAR POR PUNTOS A B Y C Y LUEGO RUIDOS DE FONDO

# Filtra los índices 3, 4, 5 de cada bloque de 6
datos_A = [x for i, x in enumerate(datos_sonometro_2) if i % 6 >= 3]

# Filtra los índices 0, 1, 2 de cada bloque de 6
datos_C = [x for i, x in enumerate(datos_sonometro_2) if i % 6 < 3]

ruido_A = np.array(datos_A[:3]) #QUITAR LAS TRES PRIMERAS POSICIONES QUE SON DE RUIDO DE FONDO
medidas_A = np.array(datos_A[3:])

ruido_B = np.array(datos_sonometro_1[:3]) #RUIDO EN B
medidas_B = np.array(datos_sonometro_1[3:]) #MEDIDAS EN B

ruido_C = np.array(datos_C[:3])
medidas_C = np.array(datos_C[3:])

#print(ruido_A, ruido_B, ruido_C) PARA COMPROBAR QUE FUNCIONE

#CORRECION RUIDO DE FONDO: PROMEDIADO DEL RUIDO DE FONDO ENERGETICO

ruido_A_promed = np.round(10 * np.log10((1/len(ruido_A)) * np.sum(10**(0.1*ruido_A))), 1)
ruido_B_promed = np.round(10 * np.log10((1/len(ruido_B)) * np.sum(10**(0.1*ruido_B))), 1)
ruido_C_promed = np.round(10 * np.log10((1/len(ruido_C)) * np.sum(10**(0.1*ruido_C))), 1)

#print(ruido_A_promed,ruido_B_promed, ruido_C_promed) #TUDO VA BEM

#print(len(medidas_A))
#AHORA VOY A SEPARAR LAS FASES

fases_A = np.split(medidas_A,7) #En este array están las medidas de cada fase, SIN PROMEDIAR
fases_B = np.split(medidas_B,7) #En este array están las medidas de cada fase, SIN PROMEDIAR
fases_C = np.split(medidas_C,7) #En este array están las medidas de cada fase, SIN PROMEDIAR


#print(fases_A[0]) Te devuelve los tres valores de la FASE 1 en la posición A

fases_A_promed = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_A
])

fases_B_promed = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_B
])

fases_C_promed = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_C
])

#TODO: COMPROBAR EL CRITERIO EN CADA FASE DE QUE EL RUIDO DE FONDO SEA 3dB INFERIOR A LA MEDIDA DE LA FASE. SE REALIZA SOBRE EL PROMEDIADO DE LAS TRES MEDIDAS EN CADA PUNTO
#EJEMPLO : FASES_A_PROMED[0]-RUIDO_A_PROMED PARA COMPROBAR EN LA FASE 1
#A INVESTIGAR ANTES: ES POSIBLE QUE HAYA QUE HACER ANTES EL PROMEDIADO DE LOS PUNTOS ABC PARA COMPROBAR ESTO CONTRA EL RUIDO ABC? LA INTUICION ME DICE QUE NO, QUE ES PUNTO A PUNTO

#Comprobación de cada punto, cada fase si cumple el criterio de los 3dB

ruido_promed = np.round(10 * np.log10((1/3) * ((10**(0.1*ruido_A_promed)) + (10**(0.1*ruido_B_promed)) + (10**(0.1*ruido_C_promed)))), 1)
fases_promed = np.round(10 * np.log10((1/3) * ((10**(0.1*fases_A_promed)) + (10**(0.1*fases_B_promed)) + (10**(0.1*fases_C_promed)))), 1)

diferencias = fases_promed - ruido_promed
print(diferencias)

#LAS FASES QUE NO CUMPLEN SON LAS 5, 6 Y 7. EN ELLAS NO SE PUEDE CORREGIR POR RUIDO DE FONDO Y LOS RESULTADOS SE DARÁN COMO COTAS

fases_1234 = fases_promed[:4]
fases_567 = fases_promed[4:7]

fases_1234_corregidas = np.round(10 * np.log10(((10**(0.1*fases_1234)) - (10**(0.1*ruido_promed)))), 1)

La_fases =  np.concatenate((fases_1234_corregidas , fases_567) , axis = 0)

print(fases_promed, La_fases)

Intentando leer el archivo: P3_G7_S1_Datos.xlsx...
Éxito: Se leyó un array de 24 filas x 1 columnas.
Intentando leer el archivo: P3_G7_S2_Datos.xlsx...


/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Éxito: Se leyó un array de 48 filas x 1 columnas.
[3.9 6.3 6.6 3.7 2.  0.5 0.9]
[43.3 45.7 46.  43.1 41.4 39.9 40.3] [41.  44.5 44.9 40.7 41.4 39.9 40.3]
